<a href="https://colab.research.google.com/github/ucfilho/optim_raianars_Insight_out_of_date/blob/master/DE_OPTIM_ANN_002_Insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################### Importar códigos

!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github

fatal: destination path 'optim_raianars_Insight' already exists and is not an empty directory.


In [0]:
%cd optim_raianars_Insight
#!ls

/content/optim_raianars_Insight


In [0]:
def AvaliaX(X):
  nrow,ncol=X.shape
  fitness = np.asarray([Fun(ind) for ind in X])
  best_idx = np.argmin(fitness)
  best = X[best_idx]
  fobj_best = fitness[best_idx]
  y=fitness
  BEST=best
  FOBEST=fobj_best
  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  x=XYsorted[:,0:ncol]
  XY=XYsorted
  BEST_XY =np.append(BEST,FOBEST)
  return XY,BEST_XY,BEST,FOBEST

In [0]:
############################### Importar bibliotecas e iniciar a contagem do tempo
import time
import matplotlib.pyplot as plt
import numpy as np
from random import randint
from scipy import optimize
import de_bounds_matriz
import de_soma_Insight
import pandas as pd
###############################

In [0]:
############################### DADOS DE ENTRADA 
RUNS=1 #NÚMERO DE RODADAS
#mut=0.8
#crossp=0.7
  ############################### Função
import Function

'''
Fun=Function.Rosenbrock
dim=30
MAX=np.repeat(10,dim)
MIN=np.repeat(-10,dim)

  #Global optimum: f(xi)= 0
'''
'''
dim=30
Fun=Function.Sphere
MAX=np.repeat(5.12,dim)
MIN=np.repeat(-5.12,dim)
  # f(x)=0 x=(0,0) [-5.12,5.12]
'''
#'''
dim=2
Fun=Function.Schwefel
MAX=np.repeat(500,dim)
MIN=np.repeat(-500,dim)
  #Global optimum: f(xi)= 0 for xi = 420.968746 for i=1,...,n  ;  xi in [-500,500]  
#'''
'''
dim=30
Fun=Function.Rastrigin
MAX=np.repeat(5.12,dim)
MIN=np.repeat(-5.12,dim)
  # f(x)=0 x=(0,0) [-5.12,5.12]
'''
'''
dim=30
Fun=Function.Sum_of_different_powers
MAX=np.repeat(1,dim)
MIN=np.repeat(-1,dim)
  # f(x)=0 x=(0,0)
'''
'''  
dim=30
Fun=Function.Ackley
MAX=np.repeat(32,dim)
MIN=np.repeat(-32,dim)
 # f(x)=0 x=(0,0) [-32, 32]
'''
'''  
dim=30
Fun=Function.Michalewicz
MAX=np.repeat(np.pi,dim)
MIN=np.repeat(0,dim)
 #fx=-9.66015 p/ d=10 [0,pi]
'''
'''  
dim=30
Fun=Function.
MAX=np.repeat(1,dim)
MIN=np.repeat(-1,dim)
'''

PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS


In [0]:
def GeraIndices(X,BESTo,FOBESTo,CRo,Fo,DIo,SOMA,TOTAL):
  QUANT=17 # quantos indices esta fazendo
  MAT_IND=np.zeros((1,QUANT))
  #MAT_INDi=np.zeros((1,QUANT))
  SOLUCOES=np.zeros((1,dimensions))
  SOLUCOESi=np.zeros((1,dimensions))

  REF=0.1 # REFERENCIA DE DIFERENCAS ENTRE OS ELEMENTOS
  nrow,ncol=X.shape
  FOBESTm=1e99
  XY,BEST_XY,BEST,FOBEST=AvaliaX(X)
  soma=0
  for j in range(ncol):
    for i in range(nrow):
        Xj=np.mean(X[:,j])
        soma=soma+(X[i,j]-Xj)**2
  # DI_ini=(soma/nrow)**0.5 a relativa precisa ser calculada fora do loop  FORA?
  DI=(soma/nrow)**0.5
  DIr=DI/DIo
  MAT_IND[0,0]=DI #dispersao
  MAT_IND[0,1]=DIr # dispersao relativa
  MAT_IND[0,2]=SOMA/TOTAL # fracao relativa
  # MAT_IND[0,3] velocidade relativa
  V1=FOBESTo
  V2=FOBEST
  A=2*V2
  if(V1 > A):
    MAT_IND[0,3]=2 # o valor de fobj torna pelo menos duas vezes melhor
  elif (V2==V1):
    MAT_IND[0,3]=0 # o valor de fobj nao altera
  else:
    MAT_IND[0,3]=1 # o valor de fobj melhora mas menos que duas vezes
  
  # MAT_IND[0,4]  # VELOC X
  DELTA=np.amax(abs(BEST-BESTo))
  if( DELTA >REF):
    MAT_IND[0,4]=2 # difere  for i in range(1,len(PARTIC)):
  elif ( DELTA == 0):
    MAT_IND[0,4]=0 # sem diferenca entre as posicoes do xbest entre duas buscas
  else:
    MAT_IND[0,4]=1 # diferenca entre as posicoes  do xbest menor que ref

  MAT_IND[0,5]=nrow  # VALOR NP
  MAT_IND[0,6]=Fo    # VALOR Fo   
  MAT_IND[0,7]=CRo   # VALOR CRo
  nrow,ncol=XY.shape
  MAT_IND[0,8]=XY[0,(ncol-1)] # VALOR fmin
  MAT_IND[0,9]=XY[(nrow-1),(ncol-1)] # VALOR fmax
  # MAT_IND[0,10] # Valor AD_fmin 
  if abs(MAT_IND[0,8]/MAT_IND[0,8])>1:
    MAT_IND[0,10]=1/abs(MAT_IND[0,8])
  else:
    MAT_IND[0,10]=MAT_IND[0,8]/MAT_IND[0,8]
  # MAT_IND[0,11] # Valor AD_fmax
  if abs(MAT_IND[0,9]/MAT_IND[0,9])>1: # tem que pegar primeira
    MAT_IND[0,11]=1/abs(MAT_IND[0,9]) # tem que pegar primeira
  else:
    MAT_IND[0,11]=MAT_IND[0,9]/MAT_IND[0,9] # tem que pegar do anterior

  # MAT_IND[0,12] # DELTA Fobj
  MAT_IND[0,12]=MAT_IND[0,3]-MAT_IND[0,3]  # tem que pegar do anterior
  

  # MAT_IND[0,13] # DELTA Vx
  MAT_IND[0,13]=MAT_IND[0,4]-MAT_IND[0,4]  # tem que pegar do anterior

  # MAT_IND[0,14] # r_fitness
  Fmin=0.05
  if (MAT_IND[0,8]==0):
    MAT_IND[0,14]=Fmin
  elif (MAT_IND[0,9]==0):
    MAT_IND[0,14]=Fmin
  else:
    if abs(MAT_IND[0,9]/MAT_IND[0,8])<1:
      MAT_IND[0,14]=1-abs(MAT_IND[0,9]/MAT_IND[0,8])
    else:
      MAT_IND[0,14]=1-abs(MAT_IND[0,8]/MAT_IND[0,9])
  # MAT_IND[0,15] # valor de F que sai da rede
  MAT_IND[0,15]=Fo
  # MAT_IND[0,16] # valor de CR que sai da rede
  MAT_IND[0,16]=CRo
  '''
  for k in range(PAR):        
    MAT_IND[k,:]=INDICE
    SOLUCOES[k,:]=BEST
    MAT_IND[k,2]=SOMA/TOTAL nao tem sentido por enquanto FORA?
    MAT_IND[k,1]=INDICE[0]/DI_ini nao tem sentido por enquanto FORA?
    MAT_IND[k,3]=FOBEST
    MAT_IND[k,8]=XY[0,(Col-1)]
    MAT_IND[k,9]=XY[(Lin-1),(Col-1)]
    DELTA=[]
    DELTA=np.amax(abs(SOLUCOES[i-1,:]-SOLUCOES[i,:]))
  
         
  #MAT_IND[k,:]=INDICE
  #SOLUCOES[k,:]=BEST
  #MAT_IND[k,2]=SOMA/TOTAL nao tem sentido por enquanto FORA?
  #MAT_IND[k,1]=INDICE[0]/DI_ini nao tem sentido por enquanto FORA?
  MAT_IND[0,3]=FOBEST
  MAT_IND[0,8]=XY[0,(ncol-1)]
  MAT_IND[0,9]=XY[(nrow-1),(ncol-1)]
  DELTA=[]
  DELTA=np.amax(abs(SOLUCOES[i-1,:]-SOLUCOES[i,:]))

  if FOBEST<FOBESTm:
    FOBESTm=np.copy(FOBEST)
    BESTm=np.copy(BEST)
    MAT_IND[0,:]=INDICE
    SOLUCOES[0,:]=BEST
    MAT_IND[0,2]=SOMA/TOTAL
    MAT_IND[0,1]=INDICE[0]/DI_ini
    MAT_IND[0,3]=FOBEST


  AUX=[]    
  AUX= np.copy(MAT_IND)  
  '''

  '''
  for i in range(1,len(PARTIC)):
    V1=AUX[i-1,3]
    V2=AUX[i,3]
    A=2*V2
    if(V1 > A):
      MAT_IND[i,3]=2 # o valor de fobj torna pelo menos duas vezes melhor
    elif (V2==V1):
      MAT_IND[i,3]=0 # o valor de fobj nao altera
    else:
      MAT_IND[i,3]=1 # o valor de fobj melhora mas menos que duas vezes
      MAT_IND[k,8]=XY[0,(Col-1)]
      MAT_IND[k,9]=XY[(Lin-1),(Col-1)]

    if abs(MAT_IND[k,9]/MAT_IND[0,9])>1:
      MAT_IND[0,11]=1/abs(MAT_IND[k,9])
    else:
      MAT_IND[0,11]=MAT_IND[k,9]/MAT_IND[0,9]

    if abs(MAT_IND[0,8]/MAT_IND[0,8])>1:
      MAT_IND[0,10]=1/abs(MAT_IND[0,8])
    else:
      MAT_IND[k,10]=MAT_IND[k,8]/MAT_IND[0,8]

  MAT_IND[0,3]=0 
  
  for k in range(len(PARTIC)):
    if abs(MAT_IND[k,9]/MAT_IND[0,9])>1:
      MAT_IND[k,11]=1/abs(MAT_IND[k,9])
    else:
      MAT_IND[k,11]=MAT_IND[k,9]/MAT_IND[0,9]

    if abs(MAT_IND[k,8]/MAT_IND[0,8])>1:
      MAT_IND[k,10]=1/abs(MAT_IND[k,8])
    else:
      MAT_IND[k,10]=MAT_IND[k,8]/MAT_IND[0,8]
    
  for i in range(1,len(PARTIC)):
    MAT_IND[i,12]=MAT_IND[i,3]-MAT_IND[i-1,3]
    MAT_IND[i,13]=MAT_IND[i,4]-MAT_IND[i-1,4]
   
    if( DELTA >REF):
      MAT_IND[i,4]=2 # difere  for i in range(1,len(PARTIC)):
      MAT_IND[i,12]=MAT_IND[i,3]-MAT_IND[i-1,3]
      MAT_IND[i,13]=MAT_IND[i,4]-MAT_IND[i-1,4]
   
     #DELTA=[]
     #DELTA=np.amax(abs(SOLUCOES[i-1,:]-SOLUCOES[i,:]))
     #print(SOLUCOES[i-1,:])
     #print(SOLUCOES[i,:])
     #print(DELTA)
      MAT_IND[i,4]=2 # diferenca entre as posicoes do xbest maior que ref
    elif ( DELTA == 0):
      MAT_IND[i,4]=0 # sem diferenca entre as posicoes do xbest entre duas buscas
    else:
      MAT_IND[i,4]=1 # diferenca entre as posicoes  do xbest menor que ref
  '''
  '''  
  MAT_IND[0,12]=0
  MAT_IND[0,13]=0
  
  for i in range(len(PARTIC)):
    MAT_IND[i,5]=int(PARTIC[i]) 
    MAT_IND[i,6]=Fi[i]
    MAT_IND[i,7]=CRi[i] 

  MAT_IND[0,3]=0

  Fmin=0.05
  for i in range(len(PARTIC)):
    if (MAT_IND[i,8]==0):
      MAT_IND[i,14]=Fmin
    elif (MAT_IND[i,9]==0):
      MAT_IND[i,14]=Fmin
    else:
      if abs(MAT_IND[i,9]/MAT_IND[i,8])<1:
        MAT_IND[i,14]=1-abs(MAT_IND[i,9]/MAT_IND[i,8])
      else:
        MAT_IND[i,14]=1-abs(MAT_IND[i,8]/MAT_IND[i,9])

  '''
  return MAT_IND



In [0]:
##################################### IMPORTAR MÓDULOS

NMAX=1000 # maximo tamanho da populacao

dimensions=len(MAX)

start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO
its=10 # quantas vezes vai usar o NPAR antes de alterar de novo
ITE=10 # PONTOS quantas vezes vai alteral NPAR 
#PARTIC=[10,80,10,400,100,100,100,100,100,100]
#ITE=50 # so para testar

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE

maxiter=3
popsize=5
p=int(TOTAL/its/ITE) 
NPAR=p
#PARTIC=np.zeros(ITE) # porque isto se eh redefinido mais em baixo e nao eh usado aqui
MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr  MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]


for i in range(RUNS):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
 
 ############################################################
  #bounds=[(5*dim, 15*dim)] * PONTOS # ITE
  #bounds=[(3*dim, 12*dim)] * PONTOS # ITE
  #bounds=[(4*dim, 16*dim)] * PONTOS
  
  # nao pode ser assim pq o bounds vale p o DE nao dos parametros
  #p=int(TOTAL/its/ITE) 
  bounds=[(p, p)] * PONTOS
  #bounds=[]
  #for k in range(dim):
  #  bounds.append((MIN[k], MAX[k]))
  # print("=========bounds=",bounds)
    


  '''
  #Geral
  new_mut=(0,1)
  new_crossp=(0,1)
  '''
  #'''
  #Multimodal Separável 
  new_mut=(0.5,1)
  new_crossp=(0,0.5)
  #'''
  '''
  #Multimodal Não Separável 
  new_mut=(0.5,1)
  new_crossp=(0.5,1)
  '''
  '''
  ##### Unimodal Separável
  new_mut=(0,0.5)
  new_crossp=(0,0.5)
  '''
  '''
  ##### Unimodal Não Separável
  new_mut=(0,0.5)
  new_crossp=(0.5,1)
  '''
 ############################################################ 
  # nao pode ser assim pq o bounds vale p o DE nao dos parametros
  ''' '''
  for j in range(PONTOS):
    bounds.append(new_mut)
    
  for j in range(PONTOS):
    bounds.append(new_crossp)

  #print("=========bounds=",bounds)
  mut=0.5;crossp=0.5;
  CRo=crossp;Fo=mut;
  #Xo=np.copy(X)
  XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X)
  DIo=1;
  nrow,ncol=X.shape
  # DIo  mut crossp FOmax FOmin Xmin Xmax
  
  MAT_IND=GeraIndices(X,BESTo,FOBESTo,CRo,Fo,DIo,SOMA,TOTAL)
  DIo=MAT_IND[0,0]

  for loop in range(ITE):
    X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,
                                                      NPAR, its,Fun,X,SOMA,TOTAL)
    MAT_IND=GeraIndices(X,BESTo,FOBESTo,CRo,Fo,DIo,SOMA,TOTAL)
    BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
    print(MAT_IND[0,14])
    Fo=0.5#MAT_IND[0,16]    # VALOR Fo   
    CRo=0.8# MAT_IND[0,17]   # VALOR CRo
    # chamar ANN calcula novos NP, CR, F 
    PARTIC=np.array([NPAR,mut,crossp])                                                       
    #print('otimo FOBEST=',FOBEST)
    #print("BEST=",BEST)
    #print("Soma= ",SOMA)                                                    
  #X,BEST,FOBEST,XY,BEST_XY,SOMA
  #  global MAX,MIN, mut, crossp, NPAR, its,Fun,X,SOMA,TOTAL,ITE,PARTIC

  '''
  print('OTIMIZAÇÃO DE_','maxiter=',maxiter,'_ popsize=', popsize,'_ bounds=',
        bounds[0],'_F',new_mut,'_Cr',new_crossp,'_ TOTAL=',TOTAL,'_ Dim=', 
        PAR,"_Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
  '''
  #print('otimo FOBEST=',FOBEST)
  #print("BEST=",BEST)
  #print("Soma= ",SOMA) 
#print("GlobalParam e Solucao:", BEST,"Função=",FOBEST)     


0.8427204182662211
0.9961701360296041
0.9937789301459683
0.9962260601624503
0.9953352747946308
0.9869046153846154
0.9944237918215614
0.05
0.05
0.05


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
XY,BEST_XY,BEST,FOBEST=AvaliaX(X)
print(BEST);print(FOBEST)
print(BEST)

[420.96874621 420.96874665]
0.0
[420.96874621 420.96874665]


In [0]:
XY.shape

(20, 3)

In [0]:
nrow,ncol=X.shape
print(nrow);print(ncol)

20
2
